In [2]:
source("utils/plot.R")

In [3]:
read_inference <- function(fname) {
    result <- read_tsv(fname, col_types=cols()) %>%
    mutate(
        filename=basename(fname),
        dirname=dirname(fname)
    ) %>%
    separate('filename', sep='_', into=c('target', 'training', NA)) %>%
    separate('dirname', sep='\\-', into=c(NA, 'dataset'))
    return(result)
}

In [4]:
data <- bind_rows(lapply(snakemake@input, read_inference)) %>%
    mutate(
        training = str_replace_all(training, c("main-fixedsweeps"="Fixed sweeps", "main-partialsweeps"="Partial sweeps")),
        training = factor(training, levels=c("Fixed sweeps", "Partial sweeps")),
        dataset = str_replace_all(dataset, c("empirical"="Genome-wide", "neutral"="Neutral simulations")),
        dataset = factor(dataset, levels=c("Genome-wide", "Neutral simulations"))
    ) %>%
    filter(!str_detect(uuid, 'sweep'))

In [5]:
selstren <- data %>%
    filter(target == "log-sel-strength")

In [6]:
selstren

In [7]:
selstren_fig <- ggplot(selstren) +
    geom_violin(aes(x=training, y=predicted_log_selection_coefficient, fill=training)) +
    scale_y_continuous(labels = function(x) {10^x}, limits=c(NA, 2)) +
    facet_wrap(vars(dataset)) +
    sweeps_fill +
    sweeps_theme +
    labs(y='Predicted s') +
    theme(
        legend.position='none',
        axis.title.x=element_blank(),
        axis.text.x=element_blank(),
        panel.grid.major.x=element_blank()
    )



In [8]:
sweepmode <- data %>%
    filter(target == "sweep-mode") %>%
    mutate(
        predicted_labels=sweepmode_factor_short(predicted_labels),
        `Trained on`=training
    )

In [9]:
sweepmode_fig <- ggplot(sweepmode) +
    geom_bar(aes(x=predicted_labels, fill=`Trained on`), position='dodge') +
    facet_wrap(vars(dataset), scales='free_y') +
    labs(y='Count') +
    sweeps_fill +
    sweeps_theme +
    theme(
        legend.position='bottom',
        axis.title.x=element_blank(),
        axis.text.y=element_blank()
    )

In [10]:
combined <- plot_grid(selstren_fig, sweepmode_fig, labels=c("A", "B"), nrow=2)
sweeps_save(snakemake@output[[1]], combined, asp=1.25)